### Bibliotecas a usar 

In [1]:
# Bibliotecas 
import pandas as pd
# Carga de geopandas
import geopandas as gpd
# Carga de folium
import folium
from folium import Marker
from folium import Circle
from folium.plugins import MarkerCluster
# Carga de plotly express
import plotly.express as px
# Carga de plotly graph objects (tabla)
import plotly.graph_objects as go
# Carga de plotly figure factory,para tipos adicionales de gráficos
import plotly.figure_factory as ff
# biblioteca para manejo de fechas
import calendar 
# Carga de math
import math

### Cargar datos a usar

In [2]:
# Cargar el csv de felinos
felinos = pd.read_csv("Datos/felinos.csv", sep="\t")
# Carga el polígono ASP
asp = gpd.read_file("Datos/ASP.geojson", sep="\t")

FileNotFoundError: [Errno 2] No such file or directory: 'Datos/felinos.csv'

### Elegir las columnas a usar

In [ ]:
# Se despliegan columnas a trabajar
felinos_columnas = felinos[["species", "stateProvince", "locality", "eventDate"]]

# Cambio de nombre de las columnas.
felinos_columnas = felinos_columnas.rename(columns={"species": "Especies",
                                              "stateProvince": "Provincia",
                                              "locality": "Localidad",
                                              "eventDate": "Fecha"})

# Cambio del tipo de datos del campo de fecha. Y que solo aparezca la fecha
felinos_columnas["Fecha"] = pd.to_datetime(felinos_columnas["Fecha"]).dt.date

# Se despliega los datos
felinos_columnas

### Eliminar fila NaN

In [ ]:
# Eliminar la filas/columnas NaN de Provincia
felinos_columnas = felinos_columnas[felinos_columnas['Provincia'].notna()]
felinos_columnas

### Elaboración de Tabla

In [ ]:
# Creación de tabla 
table = go.Figure(data=[go.Table(
    header=dict(values=list(felinos_columnas.columns),
                fill_color='mediumseagreen',
                font = dict(color = 'white', size = 12),
                align='center'),
    cells=dict(values=[felinos_columnas.Especies,
                       felinos_columnas.Provincia, 
                       felinos_columnas.Localidad, 
                       felinos_columnas.Fecha],
               fill_color='mediumaquamarine',
               font = dict(color = 'ivory', size = 11), 
               align='center'))
])

table.update_layout(width=1000, height=500)

# Mostrar tabla
table.show()

### Datos para el gráfico Sunburst

In [ ]:
# Se despliegan columnas a trabajar
felinos_genxesp = felinos[["species", "genus", "occurrenceStatus"]]

# Cambio de nombre de las columnas a nombres más claros
felinos_genxesp = felinos_genxesp.rename(columns={"species": "Especies",
                                                 "genus": "Genero",
                                                  "occurrenceStatus" :"id"})

felinos_genxesp

### Elaboración del gráfico Sunburst

In [ ]:
# Sunburst
px.sunburst(felinos_genxesp,  
            path=["Genero","Especies"],
            title="Avistamientos de felinos en Costa Rica según especie y género")

### Elaboración/Despliegue del Mapa

In [ ]:
# Creación del mapa
m = folium.Map(location=[10, -84], tiles='CartoDB positron', zoom_start=7, control_scale=True)

# Mapa Base de ESRI: NatGeo World Map
folium.TileLayer(
    tiles='http://services.arcgisonline.com/arcgis/rest/services/NatGeo_World_Map/MapServer/MapServer/tile/{z}/{y}/{x}',
    name='NatGeo World Map',
    attr='ESRI NatGeo World Map').add_to(m)

# Se añaden al mapa la capa GeoJson (ASP)
## Se añaden las ventanas emergentes al ASP

gjson = folium.GeoJson("Datos/asp.geojson", name='ASP').add_to(m)
folium.features.GeoJsonPopup(fields=['nombre_asp'],labels=False).add_to(gjson)

# Adición de datos para ventana emergente (Felinos)
felinos["popup"] = felinos["species"] + " " + felinos["stateProvince"]+ " " + felinos["eventDate"]+ " " + felinos["locality"]
felinos.fillna("NaN")

# Adición de puntos agrupados
mc = MarkerCluster(name="Registros de Felinos")
for idx, row in felinos.iterrows():
    if not math.isnan(row['decimalLongitude']) and not math.isnan(row['decimalLatitude']):
        mc.add_child(Marker([row['decimalLatitude'], row['decimalLongitude']], popup=row['popup']))


m.add_child(mc)

# Control de capas
folium.LayerControl().add_to(m)

# Despliegue del mapa
m